In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [28]:
out='../output/'
inp='../input/monthly/'
pref='fpm'

In [29]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
744,(2021/12/31) 23-24,51.50,20
745,Min,51.50,0
746,Max,170,20
747,Average,113.45766129032258064516129032,9.543010752688172043010752688


In [30]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/04/01) 04-05,39.06,9.4
6,(2017/04/01) 05-06,39.06,9.4
7,(2017/04/01) 06-07,39.06,9.4
8,(2017/04/01) 07-08,67.32,0.0
9,(2017/04/01) 08-09,67.32,0.0
...,...,...,...
740,(2021/12/31) 19-20,170.00,0.0
741,(2021/12/31) 20-21,170.00,0.0
742,(2021/12/31) 21-22,170.00,0.0
743,(2021/12/31) 22-23,51.50,20.0


In [31]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/04/01) 04-05,39.06,9.4,2017-04-01,4
6,(2017/04/01) 05-06,39.06,9.4,2017-04-01,5
7,(2017/04/01) 06-07,39.06,9.4,2017-04-01,6
8,(2017/04/01) 07-08,67.32,0.0,2017-04-01,7
9,(2017/04/01) 08-09,67.32,0.0,2017-04-01,8
...,...,...,...,...,...
740,(2021/12/31) 19-20,170.00,0.0,2021-12-31,19
741,(2021/12/31) 20-21,170.00,0.0,2021-12-31,20
742,(2021/12/31) 21-22,170.00,0.0,2021-12-31,21
743,(2021/12/31) 22-23,51.50,20.0,2021-12-31,22


In [32]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,39.06,9.4,2017-04-01,4
6,39.06,9.4,2017-04-01,5
7,39.06,9.4,2017-04-01,6
8,67.32,0.0,2017-04-01,7
9,67.32,0.0,2017-04-01,8
...,...,...,...,...
740,170.00,0.0,2021-12-31,19
741,170.00,0.0,2021-12-31,20
742,170.00,0.0,2021-12-31,21
743,51.50,20.0,2021-12-31,22


In [33]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5
2017-04-01,39.06,9.4,5,2017,4,1,5
2017-04-01,39.06,9.4,6,2017,4,1,5
2017-04-01,67.32,0.0,7,2017,4,1,5
2017-04-01,67.32,0.0,8,2017,4,1,5
...,...,...,...,...,...,...,...
2021-12-31,170.00,0.0,19,2021,12,31,4
2021-12-31,170.00,0.0,20,2021,12,31,4
2021-12-31,170.00,0.0,21,2021,12,31,4


In [34]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()
daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 4     1     122.2  4773.132            39.06      67.32      39.06   
           2     225.6  8811.936            39.06      39.06      39.06   
           3      37.6  1468.656            39.06      67.32      39.06   
           4      37.6  1468.656            39.06      67.32      39.06   
           5      75.2  2937.312            39.06      67.32      39.06   
...                ...       ...              ...        ...        ...   
2021 12    27    160.0  8240.000            51.50     170.00      51.50   
           28    160.0  8240.000            51.50     170.00      51.50   
           29     80.0  4120.000            51.50     170.00      51.50   
           30     80.0  4120.000            51.50     170.00      51.50   
           31     80.0  4120.000            51.50     170.00      51.50   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 4     1        48.951        9.4        0.0    6.110000   
           2        39.060        9.4        9.4    9.400000   
           3        61.668        9.4        0.0    1.880000   
           4        61.668        9.4        0.0    1.880000   
           5        57.900        9.4        0.0    3.133333   
...                    ...        ...        ...         ...   
2021 12    27      130.500       20.0        0.0    6.666667   
           28      130.500       20.0        0.0    6.666667   
           29      146.300       20.0        0.0    4.000000   
           30      146.300       20.0        0.0    4.000000   
           31      146.300       20.0        0.0    4.000000   

                price_morning_max  price_evening_max  price_standard  \
year month day                                                         
2017 4     1                67.32              67.32           39.06   
           2                39.06              39.06           39.06   
           3                67.32              67.32           67.32   
           4                67.32              67.32           67.32   
           5                67.32              67.32           67.32   
...                           ...                ...             ...   
2021 12    27              170.00             170.00          170.00   
           28              170.00             170.00          170.00   
           29              170.00             170.00          170.00   
           30              170.00             170.00          170.00   
           31              170.00             170.00          170.00   

                     date  
year month day             
2017 4     1   2017-04-01  
           2   2017-04-02  
           3   2017-04-03  
           4   2017-04-04  
           5   2017-04-05  
...                   ...  
2021 12    27  2021-12-27  
           28  2021-12-28  
           29  2021-12-29  
           30  2021-12-30  
           31  2021-12-31  

[1602 rows x 13 columns]

In [35]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,5,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,6,2017,4,1,5,367.164,off
2017-04-01,67.32,0.0,7,2017,4,1,5,0.000,standard
2017-04-01,67.32,0.0,8,2017,4,1,5,0.000,standard
...,...,...,...,...,...,...,...,...,...
2021-12-31,170.00,0.0,19,2021,12,31,4,0.000,standard
2021-12-31,170.00,0.0,20,2021,12,31,4,0.000,standard
2021-12-31,170.00,0.0,21,2021,12,31,4,0.000,standard


In [36]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 4     1     122.2  4773.132            39.06      67.32      39.06   
           2     225.6  8811.936            39.06      39.06      39.06   
           3      37.6  1468.656            39.06      67.32      39.06   
           4      37.6  1468.656            39.06      67.32      39.06   
           5      75.2  2937.312            39.06      67.32      39.06   
...                ...       ...              ...        ...        ...   
2021 12    27    160.0  8240.000            51.50     170.00      51.50   
           28    160.0  8240.000            51.50     170.00      51.50   
           29     80.0  4120.000            51.50     170.00      51.50   
           30     80.0  4120.000            51.50     170.00      51.50   
           31     80.0  4120.000            51.50     170.00      51.50   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 4     1        48.951        9.4        0.0    6.110000   
           2        39.060        9.4        9.4    9.400000   
           3        61.668        9.4        0.0    1.880000   
           4        61.668        9.4        0.0    1.880000   
           5        57.900        9.4        0.0    3.133333   
...                    ...        ...        ...         ...   
2021 12    27      130.500       20.0        0.0    6.666667   
           28      130.500       20.0        0.0    6.666667   
           29      146.300       20.0        0.0    4.000000   
           30      146.300       20.0        0.0    4.000000   
           31      146.300       20.0        0.0    4.000000   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2017 4     1                67.32  ...                 NaN   
           2                39.06  ...                 NaN   
           3                67.32  ...                 NaN   
           4                67.32  ...                 NaN   
           5                67.32  ...                 NaN   
...                           ...  ...                 ...   
2021 12    27              170.00  ...                 NaN   
           28              170.00  ...                 NaN   
           29              170.00  ...                 NaN   
           30              170.00  ...                 NaN   
           31              170.00  ...                 NaN   

                evening_hour_count off_volume  standard_volume  \
year month day                                                   
2017 4     1                   NaN      122.2              0.0   
           2                   NaN      225.6              NaN   
           3                   NaN       37.6              0.0   
           4                   NaN       37.6              0.0   
           5                   NaN       75.2              0.0   
...                            ...        ...              ...   
2021 12    27                  NaN      160.0              0.0   
           28                  NaN      160.0              0.0   
           29                  NaN       80.0              0.0   
           30                  NaN       80.0              0.0   
           31                  NaN       80.0              0.0   

                morning_volume  evening_volume  off_value  standard_value  \
year month day                                                              
2017 4     1               NaN             NaN   4773.132             0.0   
           2               NaN             NaN   8811.936             NaN   
           3               NaN             NaN   1468.656             0.0   
           4               NaN             NaN   1468.656             0.0   
           5               NaN             NaN   2937.312             0.0   
..

In [37]:
daily.to_csv(out + pref + '_daily.csv')

In [38]:
hourly.to_csv(out + pref + '_hourly.csv')

In [39]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-04-01,4,39.06,9.4,off
2017-04-01,5,39.06,9.4,off
2017-04-01,6,39.06,9.4,off
2017-04-01,7,67.32,0.0,standard
2017-04-01,8,67.32,0.0,standard
...,...,...,...,...
2021-12-31,19,170.00,0.0,standard
2021-12-31,20,170.00,0.0,standard
2021-12-31,21,170.00,0.0,standard


In [40]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [41]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,4124.475600,61.668,39.06,53.606050,9.400000,1.566667,4.561611,...,0.0,39.060000,39.06,NaN,NaN,NaN,9.4,NaN,0.000000,NaN
1,2017,5,107.096774,3970.077419,57.678,37.07,50.891763,10.000000,1.612903,4.634409,...,0.0,37.070000,37.07,37.07,NaN,NaN,10.0,NaN,0.404040,NaN
2,2017,6,26.400000,1176.384000,68.368,64.40,66.384000,4.400000,0.000000,1.173333,...,0.0,44.560000,44.56,NaN,NaN,NaN,4.4,NaN,0.000000,NaN
3,2017,7,273.333333,11540.133333,68.404,42.22,54.766500,20.000000,6.666667,12.333333,...,0.0,42.220000,42.22,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
4,2017,8,380.833333,22038.583333,75.400,33.40,62.785417,20.000000,15.833333,17.201389,...,0.0,57.869365,33.40,85.90,NaN,NaN,20.0,NaN,15.000000,NaN
5,2017,9,214.000000,6623.300000,56.446,30.95,48.000450,20.000000,2.666667,9.300000,...,0.0,30.950000,30.95,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
6,2017,10,214.193548,6237.316129,54.248,29.12,45.897331,20.000000,3.225806,9.317204,...,0.0,29.120000,29.12,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
7,2017,11,334.666667,12981.280000,51.628,27.26,44.207606,20.000000,11.333333,14.436111,...,0.0,38.788685,27.26,57.72,NaN,NaN,20.0,NaN,10.000000,NaN
8,2017,12,322.922581,11881.128000,51.972,26.74,43.069581,20.000000,10.438710,14.097742,...,0.0,36.792497,26.74,58.28,NaN,NaN,20.0,NaN,8.600000,NaN
9,2018,1,214.193548,6331.561290,55.256,29.56,46.794280,20.000000,3.225806,9.268817,...,0.0,29.560000,29.56,29.56,NaN,NaN,20.0,NaN,0.808081,NaN


In [42]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,4124.475600,61.668,39.06,53.606050,9.400000,1.566667,4.561611,...,0.0,39.060000,39.06,NaN,NaN,NaN,9.4,NaN,0.000000,NaN
1,2017,5,107.096774,3970.077419,57.678,37.07,50.891763,10.000000,1.612903,4.634409,...,0.0,37.070000,37.07,37.07,NaN,NaN,10.0,NaN,0.404040,NaN
2,2017,6,26.400000,1176.384000,68.368,64.40,66.384000,4.400000,0.000000,1.173333,...,0.0,44.560000,44.56,NaN,NaN,NaN,4.4,NaN,0.000000,NaN
3,2017,7,273.333333,11540.133333,68.404,42.22,54.766500,20.000000,6.666667,12.333333,...,0.0,42.220000,42.22,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
4,2017,8,380.833333,22038.583333,75.400,33.40,62.785417,20.000000,15.833333,17.201389,...,0.0,57.869365,33.40,85.90,NaN,NaN,20.0,NaN,15.000000,NaN
5,2017,9,214.000000,6623.300000,56.446,30.95,48.000450,20.000000,2.666667,9.300000,...,0.0,30.950000,30.95,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
6,2017,10,214.193548,6237.316129,54.248,29.12,45.897331,20.000000,3.225806,9.317204,...,0.0,29.120000,29.12,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
7,2017,11,334.666667,12981.280000,51.628,27.26,44.207606,20.000000,11.333333,14.436111,...,0.0,38.788685,27.26,57.72,NaN,NaN,20.0,NaN,10.000000,NaN
8,2017,12,322.922581,11881.128000,51.972,26.74,43.069581,20.000000,10.438710,14.097742,...,0.0,36.792497,26.74,58.28,NaN,NaN,20.0,NaN,8.600000,NaN
9,2018,1,214.193548,6331.561290,55.256,29.56,46.794280,20.000000,3.225806,9.268817,...,0.0,29.560000,29.56,29.56,NaN,NaN,20.0,NaN,0.808081,NaN


In [43]:
monthly=monthly.set_index(['year','month'])

In [44]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [45]:
monthly.to_csv(out + pref + '_monthly.csv')

In [46]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [47]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,price_morning_max,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,6656.532258,40.333333,84.136210,36.237500,66.463069,15.000000,2.580645,7.165323,84.136210,...,0.0,2.063525e+05,322.25,389.75,0.0,0.0,15.000,0.619674,NaN,NaN
1,2,5865.354903,43.135059,86.257345,38.559646,68.192261,12.739823,1.794690,5.949056,86.257345,...,0.0,1.656963e+05,307.25,349.75,0.0,0.0,12.675,0.000000,NaN,NaN
2,3,7060.013725,42.748169,71.041471,36.746275,57.689735,13.921569,2.745098,7.119281,71.041471,...,0.0,1.800304e+05,288.50,295.50,0.0,0.0,15.000,0.206718,NaN,NaN
3,4,6543.092453,40.932500,65.004000,37.726000,54.893316,13.880000,2.713333,6.850100,65.004000,...,0.0,1.962928e+05,320.20,371.80,0.0,0.0,13.880,0.849508,NaN,NaN
4,5,4651.633548,34.730000,70.087548,34.474000,56.939900,12.000000,2.193548,5.826882,70.087548,...,0.0,1.442006e+05,326.00,386.00,0.0,0.0,12.000,0.409595,NaN,NaN
5,6,5280.832525,33.576129,97.707131,37.741148,75.812992,14.826230,1.967213,6.549290,97.707131,...,0.0,1.288523e+05,258.80,303.60,0.0,0.0,12.880,0.000000,NaN,NaN
6,7,57877.942047,71.030821,121.514016,43.574724,93.448717,65.570079,10.866142,35.351627,121.514016,...,0.0,1.470100e+06,266.00,316.40,0.0,0.0,45.000,18.480000,NaN,NaN
7,8,5649.414615,46.304304,108.280231,43.780231,84.687486,10.461538,2.269231,5.286218,108.280231,...,0.0,1.468848e+05,275.00,321.00,0.0,0.0,12.000,3.000000,NaN,NaN
8,9,5274.014667,30.970000,75.017533,33.966000,59.997067,16.000000,2.266667,7.340000,75.017533,...,0.0,1.582204e+05,322.00,370.00,0.0,0.0,16.000,0.000000,NaN,NaN
9,10,7327.478710,32.770000,78.739806,31.804000,61.550816,20.000000,2.967742,9.282796,78.739806,...,0.0,2.271518e+05,326.00,386.00,0.0,0.0,20.000,0.000000,NaN,NaN


In [48]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [49]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                   
1     0     36.237500  15.000000   605.000000
      1     36.237500  15.000000   605.000000
      2     36.237500  15.000000   605.000000
      3     36.237500  15.000000   605.000000
      4     36.237500  15.000000   605.000000
...               ...        ...          ...
12    19    96.102258   6.077419   313.427484
      20    89.343710   8.472581   419.588452
      21    89.343710   8.472581   419.588452
      22    45.815000  22.500000  1066.500000
      23    45.815000  22.500000  1066.500000

[288 rows x 3 columns]

In [50]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [51]:
from statsmodels.tsa.stattools import pacf

In [52]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.0,1.0,1.000000
1,0.543113,NaN,NaN,0.746079
2,-0.124629,NaN,NaN,-0.192460
3,0.011240,NaN,NaN,0.264089
4,0.416062,NaN,NaN,0.008310
5,0.179135,NaN,NaN,-0.223593
6,0.077007,NaN,NaN,-0.103725
7,-0.064574,NaN,NaN,0.113945
8,0.133205,NaN,NaN,-0.237353
9,-0.045872,NaN,NaN,-0.048411


In [53]:
pacfs.to_csv(out + pref + '_pacfs.csv')